In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
!pip install bert_score datasets rouge_score evaluate transformers wandb accelerate

In [ ]:
!git clone https://github.com/Myashka/CQA_RLHF.git

In [ ]:
import os
import json
import yaml
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.13.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.13-cp38-cp38-linux_x86_64.whl

In [ ]:
!accelerate config default --mixed_precision bf16

In [ ]:
!accelerate env

In [ ]:
trainer_config = dict(
    model_name = 'EleutherAI/gpt-neo-125M',
    seed = 42,
    batch_size = 16,
    max_length = 512,
    learning_rate = 5e-5,
    n_epoches = 3,
    eval_every = 1000,
    warmup_steps = 100,
    wandb_kwargs = dict(
        entity = 'myashka',
        job_type = 'train',
        group = 'sft'
    ),
    TPU = True,
    gradient_accumulation_steps = 1,
    freeze = True,
    use_cache = False,
    wandb_api = 'text'
)

with open('trainer_config.yaml', 'w') as outfile:
    yaml.dump(trainer_config, outfile, default_flow_style=False)

In [ ]:
ds_config = {
    "bf16": {"enabled": True},
    "zero_optimization": {
        "stage": 3,
        "stage3_gather_16bit_weights_on_model_save": False,
        "offload_optimizer": {"device": None},
        "offload_param": {"device": None},
    },
    "gradient_clipping": 1.0,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": 1,
    "steps_per_print": 2000000,
}

with open('ds-config.json', 'w') as f:
    json.dump(ds_config, f)


In [ ]:
accelerate_config = dict(
    compute_environment = 'LOCAL_MACHINE',
    deepspeed_config = dict(
        deepspeed_config_file = 'c',
        zero3_init_flag = True
    ),
    distributed_type = 'DEEPSPEED',
    fsdp_config = dict(),
    machine_rank = 0,
    main_process_ip = None,
    main_process_port = None,
    main_training_function = 'main',
    mixed_precision = 'bf16',
    num_machines = 1,
    num_processes = 8,
    use_cpu = False

)

with open('accelerate_config.yaml', 'w') as outfile:
    yaml.dump(accelerate_config, outfile, default_flow_style=False)

In [ ]:
!mv accelerate_config.yaml 

In [ ]:
mixed_precision = 'bf16'
output_dir = r'/content/drive/MyDrive/Diploma/Checkpoints'
data_path = r'/content/drive/MyDrive/Diploma/data/1.0-data-div-ans-sep-test.json'

num_processes = '8'
config_file = r'/content/CQA_RLHF/sft/accelerate/config.yaml'

In [ ]:
!accelerate launch --tpu --downcast_bf16 --mixed_precision $mixed_precision /content/CQA_RLHF/sft/accelerate/train_accelerate.py --mixed_precision $mixed_precision --output_dir $output_dir --data_path $data_path --config_file $config_file